In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import PIL
import PIL.Image

In [ ]:
#load the training and testing datasets
training_dir = './cleaned/Training'
testing_dir = './cleaned/Testing'
training_set = tf.keras.utils.image_dataset_from_directory(training_dir)
testing_set = tf.keras.utils.image_dataset_from_directory(testing_dir)

class_names = training_set.class_names
print(class_names)


    

In [ ]:
#now we are doing to do some data visualization

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in training_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')



        
    
    

In [ ]:
#now we are going to standardize the data in order to make it more compatible for a neural network
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_training_set = training_set.map(lambda x, y: (normalization_layer(x), y))
normalized_testing_set = testing_set.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
training_images, training_labels = zip(*normalized_training_set)
testing_images, testing_labels = zip(*normalized_testing_set)
plt.figure(figsize=(10, 10))
for i in range(2):
    plt.subplot(5, 5, i + 1)
    plt.grid = False
    plt.imshow(training_images[i][0])
    plt.xlabel(class_names[training_labels[i][0]])
plt.show()
training_images = np.concatenate(training_images)
training_labels = np.concatenate(training_labels)
testing_images = np.concatenate(testing_images)
testing_labels = np.concatenate(testing_labels)


In [ ]:
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

In [ ]:
#adding layers to our sequential model
from tensorflow.keras import models, layers
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.summary()

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(class_names)))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(training_images, training_labels, epochs=10, 
                    validation_data=(testing_images, testing_labels))